# Arithmetic Operations And Aggregations


Import the LArray library:


In [ ]:
from larray import *

## Arithmetic operations


In [ ]:
# load the 'demography_eurostat' dataset
demography_eurostat = load_example_data('demography_eurostat')

# extract the 'population' array
population = demography_eurostat.population

# show the 'population' array
population

In [ ]:
# let us also imagine a constant growth rate of the population
constant_growth_rate = Array(data=[[1.011, 1.010], [1.013, 1.011], [1.010, 1.009]], 
                             axes=population.axes - population.time)
constant_growth_rate

One can do all usual arithmetic operations on an array, it will apply the operation to all elements individually


In [ ]:
# 'true' division
population_in_millions = population / (10**6)
population_in_millions

In [ ]:
# 'floor' division
population_in_millions = population // (10**6)
population_in_millions

<div class="alert alert-warning">
**Warning:** Python has two different division operators: 

- the 'true' division (/) always returns a float.
- the 'floor' division (//) returns an integer result (discarding any fractional result).
</div>

In [ ]:
# % means modulo (aka remainder of division)
population % (10**6)

In [ ]:
# ** means raising to the power
constant_growth_rate ** 3

More interestingly, binary operators as above also works between two arrays:


In [ ]:
# first, we store the population of the year 2017 in a new variable
population_2017 = population[2017]
population_2017

In [ ]:
# perform an arithmetic operation between two arrays
population_2020 = population_2017 * (constant_growth_rate ** 3)
population_2020

<div class="alert alert-info">
**Note:** Be careful when mixing different data types.
You can use the method [astype](../_generated/larray.Array.astype.rst#larray.Array.astype) to change the data type of an array.
</div>


In [ ]:
# force the resulting matrix to be an integer matrix
population_2020 = (population_2017 * (constant_growth_rate ** 3)).astype(int)
population_2020

### Axis order does not matter much (except for output)

You can do operations between arrays having different axes order.
The axis order of the result is the same as the left array


In [ ]:
# let's change the order of axes of the 'constant_growth_rate' array
transposed_growth_rate = constant_growth_rate.transpose()

# look at the order of the new 'transposed_growth_rate' array:
# 'gender' is the first axis while 'country' is the second
transposed_growth_rate

In [ ]:
# look at the order of the 'population_2017' array:
# 'country' is the first axis while 'gender' is the second
population_2017

In [ ]:
# LArray doesn't care of axes order when performing 
# arithmetic operations between arrays
population_2018 = population_2017 * transposed_growth_rate
population_2018

### Axes must be compatible

Arithmetic operations between two arrays only works when they have compatible axes (i.e. same list of labels in the same order).

In [ ]:
# show 'population_2017'
population_2017

#### Order of labels matter

In [ ]:
# let us imagine that the labels of the 'country' axis 
# of the 'constant_growth_rate' array are in a different order
# than in the 'population_2017' array
reordered_growth_rate = constant_growth_rate.reindex('country', ['Germany', 'Belgium', 'France'])
reordered_growth_rate

In [ ]:
# when doing arithmetic operations, 
# the order of labels counts
try:
    population_2018 = population_2017 * reordered_growth_rate
except Exception as e:
    print(type(e).__name__, e)

#### No extra or missing labels are permitted

In [ ]:
# let us imagine that the 'country' axis of 
# the 'constant_growth_rate' array has an extra 
# label 'Netherlands' compared to the same axis of 
# the 'population_2017' array
growth_rate_Netherlands = Array([1.012, 1.], population.gender)
growth_rate_extra_country = constant_growth_rate.append('country', growth_rate_Netherlands, label='Netherlands')
growth_rate_extra_country

In [ ]:
# when doing arithmetic operations, 
# no extra or missing labels are permitted 
try:
    population_2018 = population_2017 * growth_rate_extra_country
except Exception as e:
    print(type(e).__name__, e)

#### Ignoring labels (risky)

<div class="alert alert-warning">
    **Warning:** Operations between two arrays only works when they have compatible axes (i.e. same labels) but this behavior can be override via the [ignore_labels](../_generated/larray.Array.ignore_labels.rst#larray.Array.ignore_labels) method.
In that case only the position on the axis is used and not the labels.

Using this method is done at your own risk and SHOULD NEVER BEEN USED IN A MODEL. 
Use this method only for quick tests or rapid data exploration. 
</div>


In [ ]:
# let us imagine that the labels of the 'country' axis 
# of the 'constant_growth_rate' array are the 
# country codes instead of the country full names
growth_rate_country_codes = constant_growth_rate.set_labels('country', ['BE', 'FR', 'DE'])
growth_rate_country_codes

In [ ]:
# use the .ignore_labels() method on axis 'country'
# to avoid the incompatible axes error (risky)
population_2018 = population_2017 * growth_rate_country_codes.ignore_labels('country')
population_2018

### Extra Or Missing Axes (Broadcasting)

The condition that axes must be compatible only applies on common axes. 
Arithmetic operations between two arrays can be performed even if the second array has extra or missing axes compared to the first one:

In [ ]:
# let's define a 'multiplicator' vector with 
# one value defined for each gender
multiplicator = Array([-1, 1], axes=population.gender)
multiplicator

In [ ]:
# the multiplication below has been propagated to the 
# 'country' and 'time' axes.
# This behavior is called broadcasting
population * multiplicator

### Boolean Operations

Python comparison operators are: 

| Operator  | Meaning                 |
|-----------|-------------------------|
|``==``     | equal                   | 
|``!=``     | not equal               | 
|``>``      | greater than            | 
|``>=``     | greater than or equal   | 
|``<``      | less than               | 
|``<=``     | less than or equal      |

Applying a comparison operator on an array returns a boolean array:

In [ ]:
# test which values are greater than 10 millions
population > 10e6

Comparison operations can be combined using Python bitwise operators:

| Operator | Meaning                              |
|----------|------------------------------------- |
|  &       | and                                  |
|  \|      | or                                   |
|  ~       | not                                  |

In [ ]:
# test which values are greater than 10 millions and less than 40 millions
(population > 10e6) & (population < 40e6)

In [ ]:
# test which values are less than 10 millions or greater than 40 millions
(population < 10e6) | (population > 40e6)

In [ ]:
# test which values are not less than 10 millions
~(population < 10e6)

The returned boolean array can then be used in selections and assignments:

In [ ]:
population_copy = population.copy()

# set all values greater than 40 millions to 40 millions
population_copy[population_copy > 40e6] = 40e6
population_copy

Boolean operations can be made between arrays:

In [ ]:
# test where the two arrays have the same values
population == population_copy

To test if all values between are equals, use the [equals](../_generated/larray.Array.equals.rst#larray.Array.equals) method:

In [ ]:
population.equals(population_copy)

## Aggregates

The LArray library provides many aggregation functions. The list is given in the [Aggregation Functions](../api.rst#aggregation-functions) subsection of the [API Reference](../api.rst) page.

Aggregation operations can be performed on axes or groups. Axes and groups can be mixed. 

The main rules are: 

-  Axes are separated by commas ``,``
-  Groups belonging to the same axis are grouped inside parentheses ()


Calculate the sum along an axis:

In [ ]:
population.sum('gender')

or several axes (axes are separated by commas ``,``):

In [ ]:
population.sum('country', 'gender')

Calculate the sum along all axes except one by appending `_by` to the aggregation function:


In [ ]:
population.sum_by('time')

Calculate the sum along groups (the groups belonging to the same axis must grouped inside parentheses ()):


In [ ]:
even_years = population.time[2014::2] >> 'even_years'
odd_years = population.time[2013::2] >> 'odd_years'

population.sum((odd_years, even_years))

Mixing axes and groups in aggregations:


In [ ]:
population.sum('gender', (odd_years, even_years))